In [1]:
import warnings
import os
from glob import glob
from pathlib import Path
import operator

import pandas as pd
import polars as pl
import polars.selectors as cs
from sklearn import preprocessing
from category_encoders import OneHotEncoder
from category_encoders.target_encoder import TargetEncoder

pd.set_option('display.float_format',lambda x : '%.6f' % x)
warnings.filterwarnings('ignore')

In [2]:
def set_table_dtypes(df):
    for col in df.columns:
        if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2", "target"]:
            df = df.with_columns(pl.col(col).cast(pl.Int64))
        elif col in ["date_decision"]:
            df = df.with_columns(pl.col(col).cast(pl.Date))
        elif col[-1] in ("P", "A"):
            df = df.with_columns(pl.col(col).cast(pl.Float64))
        elif col[-1] in ("M",):
            df = df.with_columns(pl.col(col).cast(pl.String))
        elif col[-1] in ("D",):
            df = df.with_columns(pl.col(col).cast(pl.Date))

    return df

In [3]:
def read_file(path, depth=None):
    df = pl.read_parquet(path)
    df = df.pipe(set_table_dtypes)
    return df

def read_files(regex_path, depth=None):
    chunks = []
    for path in glob(str(regex_path)):
        chunks.append(pl.read_parquet(path).pipe(set_table_dtypes))

    df = pl.concat(chunks, how="vertical_relaxed")
    return df

In [4]:
train_dir_path = "/Users/sophie/workspace/data/kaggle/home-credit-credit-risk-model-stability/parquet_files/train"
test_dir_path = "/Users/sophie/workspace/data/kaggle/home-credit-credit-risk-model-stability/parquet_files/test"
train_file = 'train_debitcard_1.parquet'
test_file = 'test_debitcard_1.parquet'
train_file_path = Path(os.path.join(train_dir_path, train_file))
test_file_path = Path(os.path.join(test_dir_path, test_file))

target_file = 'train_base.parquet'
target_file_path = Path(os.path.join(train_dir_path, target_file))
target_file_test = 'test_base.parquet'
target_file_test_path = Path(os.path.join(test_dir_path, target_file_test))

In [5]:
df_base_train = read_file(target_file_path)
df_base_train = df_base_train.with_columns(IS_TRAIN = pl.lit(1))
df_train = read_file(train_file_path)
df_train = df_base_train.join(df_train, how="left", on="case_id")

In [6]:
df_base_test = read_file(target_file_test_path)
df_base_test = df_base_test.with_columns(target = None)
df_base_test = df_base_test.with_columns(IS_TRAIN = pl.lit(0))
df_test = read_file(test_file_path)
df_test = df_base_test.join(df_test, how="left", on="case_id")

In [35]:
data = pl.concat([df_train, df_test], how="vertical_relaxed")

In [36]:
data_base = pl.concat([df_base_train, df_base_test], how="vertical_relaxed")

In [28]:
del df_train, df_test

In [37]:
data.shape

(1572199, 11)

In [ ]:
# null的比较多

In [38]:
data.head(50)

case_id,date_decision,MONTH,WEEK_NUM,target,IS_TRAIN,last180dayaveragebalance_704A,last180dayturnover_1134A,last30dayturnover_651A,num_group1,openingdate_857D
i64,date,i64,i64,i64,i32,f64,f64,f64,i64,date
0,2019-01-03,201901,0,0,1,null,null,null,null,null
1,2019-01-03,201901,0,0,1,null,null,null,null,null
2,2019-01-04,201901,0,0,1,null,null,null,null,null
3,2019-01-03,201901,0,0,1,null,null,null,null,null
4,2019-01-04,201901,0,1,1,null,null,null,null,null
5,2019-01-02,201901,0,0,1,null,null,null,null,null
6,2019-01-03,201901,0,0,1,null,null,null,null,null
7,2019-01-03,201901,0,0,1,null,null,null,null,null
8,2019-01-03,201901,0,0,1,null,null,null,null,null


In [39]:
# df_train.filter(pl.col("case_id") == 57633)
# data.select(['case_id', 'num_group1', 'target']).filter(pl.col('num_group1').is_not_null())
data.filter(pl.col('num_group1').is_not_null())
# df_train.shape

case_id,date_decision,MONTH,WEEK_NUM,target,IS_TRAIN,last180dayaveragebalance_704A,last180dayturnover_1134A,last30dayturnover_651A,num_group1,openingdate_857D
i64,date,i64,i64,i64,i32,f64,f64,f64,i64,date
225,2019-01-04,201901,0,0,1,null,null,null,0,2016-08-16
331,2019-01-05,201901,0,0,1,null,null,null,0,2015-03-19
358,2019-01-05,201901,0,0,1,null,null,null,0,2014-09-02
390,2019-01-07,201901,0,0,1,null,null,null,0,2014-07-23
390,2019-01-07,201901,0,0,1,null,null,null,2,2016-06-08
390,2019-01-07,201901,0,0,1,null,null,null,1,2015-10-01
445,2019-01-08,201901,1,0,1,null,null,null,4,2016-07-19
445,2019-01-08,201901,1,0,1,null,null,null,1,2015-08-01
445,2019-01-08,201901,1,0,1,null,null,null,3,2016-02-22


In [40]:
data = data.sort('case_id', 'num_group1')

In [41]:
class Aggregator:
    @staticmethod
    def num_expr(df):
        cols = [col for col in df.columns if col[-1] in ("P", "A")]

        expr_num = []
        # expr_num.extend([pl.max(col).alias(f"max_{col}") for col in cols])
        # expr_num.extend([pl.min(col).alias(f"min_{col}") for col in cols])
        # expr_num.extend([pl.mean(col).alias(f"mean_{col}") for col in cols])
        # expr_num.extend([pl.count(col).alias(f"count_{col}") for col in cols])
        # expr_num.extend([pl.var(col).alias(f"var_{col}") for col in cols])
        expr_num.extend([pl.last(col).alias(f"last_{col}") for col in cols])
        # expr_num.extend([pl.first(col).alias(f"first_{col}") for col in cols])

        return expr_num

    @staticmethod
    def date_expr(df):
        cols = [col for col in df.columns if col[-1] in ("D",)]

        expr_date = []
        expr_date.extend([pl.max(col).alias(f"max_{col}") for col in cols])
        expr_date.extend([pl.min(col).alias(f"min_{col}") for col in cols])

        return expr_date

    @staticmethod
    def str_expr(df):
        cols = [col for col in df.columns if col[-1] in ("M",)]
        cols.remove('WEEK_NUM')

        expr_str = []
        expr_str.extend([pl.last(col).alias(f"last_{col}") for col in cols])
        expr_str.extend([pl.first(col).alias(f"first_{col}") for col in cols])
        expr_str.extend([pl.n_unique(col).alias(f"n_unique_{col}") for col in cols])

        return expr_str

    @staticmethod
    def other_expr(df):
        cols = [col for col in df.columns if col[-1] in ("T", "L")]

        expr_other = []
        expr_other.extend([pl.max(col).alias(f"max_{col}") for col in cols])
        expr_other.extend([pl.min(col).alias(f"min_{col}") for col in cols])
        expr_other.extend([pl.mean(col).alias(f"mean_{col}") for col in cols])
        expr_other.extend([pl.var(col).alias(f"var_{col}") for col in cols])
        expr_other.extend([pl.last(col).alias(f"last_{col}") for col in cols])
        expr_other.extend([pl.first(col).alias(f"first_{col}") for col in cols])

        return expr_other

    @staticmethod
    def count_expr(df):
        cols = [col for col in df.columns if "num_group" in col]

        expr_count = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_count

    @staticmethod
    def get_exprs(df):
        exprs = Aggregator.num_expr(df) + \
                Aggregator.date_expr(df) + \
                Aggregator.str_expr(df) + \
                Aggregator.other_expr(df) + \
                Aggregator.count_expr(df)

        return exprs

In [42]:
fea_agg = data.group_by("case_id").agg(Aggregator.get_exprs(data))

In [43]:
fea_agg.shape

(1526669, 7)

In [44]:
fea_agg.head(20)

case_id,last_last180dayaveragebalance_704A,last_last180dayturnover_1134A,last_last30dayturnover_651A,max_openingdate_857D,min_openingdate_857D,max_num_group1
i64,f64,f64,f64,date,date,i64
0,null,null,null,null,null,null
1,null,null,null,null,null,null
2,null,null,null,null,null,null
3,null,null,null,null,null,null
4,null,null,null,null,null,null
5,null,null,null,null,null,null
6,null,null,null,null,null,null
7,null,null,null,null,null,null
8,null,null,null,null,null,null


In [45]:
data = data_base.join(fea_agg, how="left", on="case_id")

In [59]:
data.head(10)

case_id,date_decision,MONTH,WEEK_NUM,target,IS_TRAIN,last_last180dayaveragebalance_704A,last_last180dayturnover_1134A,last_last30dayturnover_651A,max_openingdate_857D,min_openingdate_857D,max_num_group1
i64,date,i64,i64,i64,i32,i32,i32,i32,date,date,i64
0,2019-01-03,201901,0,0,1,0,0,0,null,null,null
1,2019-01-03,201901,0,0,1,0,0,0,null,null,null
2,2019-01-04,201901,0,0,1,0,0,0,null,null,null
3,2019-01-03,201901,0,0,1,0,0,0,null,null,null
4,2019-01-04,201901,0,1,1,0,0,0,null,null,null
5,2019-01-02,201901,0,0,1,0,0,0,null,null,null
6,2019-01-03,201901,0,0,1,0,0,0,null,null,null
7,2019-01-03,201901,0,0,1,0,0,0,null,null,null
8,2019-01-03,201901,0,0,1,0,0,0,null,null,null


In [55]:
data = data.with_columns( pl.when(data['last_last180dayaveragebalance_704A'].is_null()).then(0).otherwise(1).alias('last_last180dayaveragebalance_704A') )

In [57]:
data = data.with_columns( pl.when(data['last_last180dayturnover_1134A'].is_null()).then(0).otherwise(1).alias('last_last180dayturnover_1134A') )

In [58]:
data = data.with_columns( pl.when(data['last_last30dayturnover_651A'].is_null()).then(0).otherwise(1).alias('last_last30dayturnover_651A') )

In [14]:
# derive_list = [('last180dayturnover_1134A', 'last30dayturnover_651A'), ('last180dayturnover_1134A', 'last180dayaveragebalance_704A')]

In [15]:
# def derived_features_between_cols(df, cols_list):
#     for item in cols_list:
#         print(item)
#         col1 = item[0]
#         col2 = item[1]
#
#         df = df.with_columns((pl.col(col2) - pl.col(col1)).alias(col2 + '_minus_' + col1))
#         df = df.with_columns(operator.truediv(pl.col(col2), pl.col(col1)).alias(col2 + '_div_' + col1))
#
#     return df
#
# data = derived_features_between_cols(data, derive_list)

('last180dayturnover_1134A', 'last30dayturnover_651A')
('last180dayturnover_1134A', 'last180dayaveragebalance_704A')


In [25]:
# def filter_cols(df):
#     dt_train = df.filter(pl.col('IS_TRAIN') == 1)
#     for col in df.columns:
#         if col not in ["case_id", "WEEK_NUM", "date_decision", "MONTH", "IS_TRAIN", "target"]:
#             isnull = dt_train[col].is_null().mean()
#
#             if isnull > 0.95:
#                 print(f'col [{col}] to be null dropped...')
#                 num_positive = dt_train.filter(pl.col(col).is_not_null() & pl.col("target") == 1).shape[0]
#                 if num_positive < 1000:
#                     df = df.drop(col)
#                     print(f'col [{col}] null dropped...')
#
#     for col in df.columns:
#         if (col not in ["case_id", "WEEK_NUM", "date_decision", "MONTH", "IS_TRAIN", "target"]) & (df[col].dtype == pl.String):
#             freq = dt_train[col].n_unique()
#
#             if freq == 1:
#                 df = df.drop(col)
#                 print(f'col [{col}] freq dropped...')
#
#     return df

In [26]:
# data = data.pipe(filter_cols)

col [mean_last180dayaveragebalance_704A] to be null dropped...
col [mean_last180dayaveragebalance_704A] null dropped...
col [mean_last180dayturnover_1134A] to be null dropped...
col [mean_last180dayturnover_1134A] null dropped...
col [mean_last30dayturnover_651A] to be null dropped...
col [mean_last30dayturnover_651A] null dropped...
col [mean_last30dayturnover_651A_minus_last180dayturnover_1134A] to be null dropped...
col [mean_last30dayturnover_651A_minus_last180dayturnover_1134A] null dropped...
col [mean_last30dayturnover_651A_div_last180dayturnover_1134A] to be null dropped...
col [mean_last30dayturnover_651A_div_last180dayturnover_1134A] null dropped...
col [mean_last180dayaveragebalance_704A_minus_last180dayturnover_1134A] to be null dropped...
col [mean_last180dayaveragebalance_704A_minus_last180dayturnover_1134A] null dropped...
col [mean_last180dayaveragebalance_704A_div_last180dayturnover_1134A] to be null dropped...
col [mean_last180dayaveragebalance_704A_div_last180dayturn

In [60]:
data.shape

(1526669, 12)

In [61]:
def handle_date(df):
    for col in df.columns:
        if col[-1] in ("D",) and df[col].dtype == pl.Date:
            print(f'col [{col}] is date...')
            df = df.with_columns(pl.col(col) - pl.col("date_decision"))
            df = df.with_columns(pl.col(col).dt.total_days())

    # df = df.drop("date_decision", "MONTH")
    return df


def handle_category(df):
    dt_train = df.filter(pl.col('IS_TRAIN') == 1)
    cols_list = df.columns
    for col in cols_list:
        if ((col[-1] in ("M",)) and ('WEEK_NUM' not in col) and (not col.startswith('n_unique'))) \
                or (col[-1] in ("L", "P") and (df[col].dtype == pl.String)):
            freq = dt_train[col].n_unique()
            if freq <= 10:
                print(f'col [{col}] one hot encoded...')
                encoder = OneHotEncoder(cols=[col], handle_unknown='indicator', handle_missing='indicator', use_cat_names=True)
                encoder.fit(dt_train[col].to_pandas())
                res_df = pl.from_pandas(encoder.transform(df[col].to_pandas()))
                df = pl.concat([df, res_df], how="horizontal")
                df = df.drop(col)
            else:
                print(f'col [{col}] target encoded...')
                encoder = TargetEncoder(cols=[col], handle_unknown='value', handle_missing='value') \
                            .fit(dt_train[col].to_pandas(), dt_train['target'].to_pandas())
                df = df.with_columns(pl.from_pandas(encoder.transform(df[col].to_pandas())).to_series().alias(col + '_target_encoder'))
                df = df.drop(col)

    return df

In [29]:
# data = data.pipe(handle_category)

In [62]:
data = data.pipe(handle_date)

col [max_openingdate_857D] is date...
col [min_openingdate_857D] is date...


In [65]:
data.head(10)

case_id,IS_TRAIN,last_last180dayaveragebalance_704A,last_last180dayturnover_1134A,last_last30dayturnover_651A,max_openingdate_857D,min_openingdate_857D,max_num_group1
i64,i32,i32,i32,i32,i64,i64,i64
0,1,0,0,0,null,null,null
1,1,0,0,0,null,null,null
2,1,0,0,0,null,null,null
3,1,0,0,0,null,null,null
4,1,0,0,0,null,null,null
5,1,0,0,0,null,null,null
6,1,0,0,0,null,null,null
7,1,0,0,0,null,null,null
8,1,0,0,0,null,null,null


In [64]:
data = data.drop('date_decision', 'MONTH', 'WEEK_NUM', 'target')

In [66]:
data.shape

(1526669, 8)

In [67]:
preprocess_dir_path = "/Users/sophie/workspace/data/kaggle/home-credit-credit-risk-model-stability/preprocess"
preprocess_file = 'debitcard_1.parquet'
preprocess_file_path = Path(os.path.join(preprocess_dir_path, preprocess_file))
data.write_parquet(preprocess_file_path)